In [21]:
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import random
import pickle
import sys
import numpy as np
added_path = '../src'
if added_path not in sys.path:
    sys.path.append(added_path)

import components.data_transformation as data_transformation

def load_model():
    with open("../models/best_model.pkl", 'rb') as f:
        return pickle.load(f)
    
class segmentator:
    def __init__(self):
       self.pipeline = None

    def process(self, image):
        return  0, {'area': 0, 'perimeter': 0, 'eccentricity': 0, 'Equivalent Diameter': 0} 

class model:
    def __init__(self):
        self.model = load_model()
        channel_features = ['mean', 'std', 'median', 'mode', 'min', 'max', 'range', 'skewness', 'kurtosis', 'entropy', 'quantile_0.25', 'quantile_0.75', 'iqr']
        histogram_features = ['mean', 'std', 'median', 'mode', 'min', 'max', 'range', 'skewness', 'kurtosis', 'entropy', 'R']
        coocurrence_matrix_features = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        
        # Data Transformation
        self.transformation_config = data_transformation.DataTransformationConfig(
            channel_features=channel_features,
            histogram_features=histogram_features,
            coocurrence_matrix_features=coocurrence_matrix_features,
            use_augmentation= 0,
            positive_class="pools",
            negative_class="no_pools",
            to_grayscale=self.to_grayscale,
            to_histogram=self.to_histogram,
        )

        self.transformer = data_transformation.DataTransformer(self.transformation_config)

    def to_grayscale(images: list):
        grayscale_images = []
        for image in images:
            grayscale_image = np.array(Image.fromarray(image).convert('L'))
            grayscale_images.append(grayscale_image)
        return grayscale_images

    def to_histogram(image_gray: np.ndarray, bins: int = 256):
        hist, _ = np.histogram(image_gray, bins=bins, density=True)
        return hist
    
    def predict(self, image):
        features = self.transformer.transform_single_image(image)
        return self.model.predict(features)


def process_image(image, model: model, segmentator: segmentator):
    print(image.shape)
    division_x = 7
    division_y = 8
    m = image.shape[0] // division_x
    n = image.shape[1] // division_y 

    resultant_image = image.copy()
    frame = pd.DataFrame(columns=['label', 'area', 'perimeter', 'eccentricity', 'Equivalent Diameter'])
    label = 0
    for i in range(division_x):
        for j in range(division_y):
            sub_image = image[int(i*m):int((i+1)*m), int(j*n):int((j+1)*n)]
            if model.predict(sub_image):
                segmented_sub_image, caracteristicas = segmentator.process(sub_image)
                caracteristicas['label'] = label
                frame.loc[len(frame)] = caracteristicas
                resultant_image[int(i*m):int((i+1)*m), int(j*n):int((j+1)*n)] = segmented_sub_image
                label += 1
    return resultant_image, frame



imag = cv2.imread(os.path.join('..', 'image_chunk.jpg'))
image_rgb = cv2.cvtColor(imag, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10,10))
plt.imshow(image_rgb)
plt.title('Original')
plt.show()

res = process_image(imag, model(), segmentator())

plt.figure(figsize=(10,10))
plt.imshow(res[0])
plt.title('Segmented')
plt.show()

print(res[1].to_string(index=False))

SyntaxError: invalid syntax (1270337317.py, line 59)